In [6]:
import pandas as pd
import numpy as np
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

In [7]:
def binomial (n, p, size):
    """
    Generate random numbers following a binomial distribution.

    Parameters:
    - n:    Number of trials (number of times an event is repeated).
    - p:    Probability of success for each trial.
    - size: Number of times to repeat the binomial experiment.

    Returns:
    - An array of random numbers representing the number of successes in each experiment.
    """
    return np.random.binomial(n, p, size)

def randint (low, high, size):
    """
    Generate random integers from a discrete uniform distribution.

    Parameters:
    - low:  The minimum (inclusive) value of the range.
    - high: The maximum (exclusive) value of the range.
    - size: The number of random integers to generate.

    Returns:
    - An array of random integers within the specified range.
    """
    return np.random.randint (low, high, size)

# QUESTION 1
There are n products to be produced $\newline$
There are m suppliers which provide the parts to produce the products $\newline$
A product i needs $a_{ij}$ parts ($a_{ij >= 0}$, $i = 1\rightarrow n$, $j=1\rightarrow m $) $\newline$
The preorder cost for part $j$ is $b_j$ $\newline$ 
The demand for the products is random vector $D = (D_1,..., D_n)$ $\newline$
Cost additionaly $l_i$ to satisfy a unit of demand for product $i$, and unit selling price of that unit is $q_i$ $\newline$
The number of parts ordered is $x_j$, the number of units produced is $z_i$, the numbers of remaining parts unused is $y_j$


### Second stage
The second stage is mentioned first to emphasize the importance of understanding the uncertainty in case of solving a stochastic problem. This approach aligns with the principles of stochastic programming, where the second-stage decisions are made after observing the realization of random data and are contingent on the first-stage decisions. $\newline$
In this stage, we want to minimize the cost of production. Specifically, we need to find the optimal cost of filling an additional portion of demand while reselling the salvage parts to minimize the loss of not using the parts. $\newline$
$$ 
min_{x,y} \text { } Z=\sum^n_{i=1} (l_i-q_i)z_i - \sum^m_{j=1}s_jy_j \newline \text {subject to} \newline
y_j=x_j- \sum_{i=1}^n a_{ij}z_i, j=1,...,m \newline
0 \leq z_i \leq d_i, i=1,...,n \newline
y_j \geq 0, j=1,...,m
$$

### First stage
In this stage, $x$ needs to be decided before a realization of the demand $D$, which makes it a $\textit {here and now}$ decisions $\newline$
Moreover, the expected value of $Z$ should also be taken into account. To be specific, that is the optimal value $Q(x)$ of the second stage ($Q(x) = E[Z(z,y)]$) $\newline$
Therefore, we yield this problem below: 
$$min \text { } g(x,y,z) = b^Tx + Q(x) = b^Tx + E[Z(z,y)]$$

### Big problem
We need to solve this
$$
min \text { } b^Tx + \sum^S_{i=1} p_s[(l_i-q_i)^Tz_i - s^T_iy_i] \newline
\text {subject to} \newline
y \geq 0 \newline
0 \leq z \leq d \newline
x \geq 0 \newline
$$

In [8]:
n   = 8           # number of products
m   = 5           # number of parts to be ordered
S   = 2           # number of scenarios
p_s = [0.5, 0.5]  # probability of each scenario / density

D = np.array([binomial (10, 0.5, n),
              binomial (10, 0.5, n)]) # demand for each product in each scenario matrix [n x S]

b = randint(5, 10, m) # preorder cost for each part

A = []
for i in range(n):
    A.append(np.array(randint(1,10,m)))

A = np.array(A) # bill of materials matrix [m x n]

s = randint (1, 10, m)       # salvage values
l = randint (100, 200, n)    # additional costs
q = randint (1000, 1400, n)  # unit selling prices

c = l - q #cost coefficients

In [9]:
print('Preorder cost: ', b)
print('Material usage:\n', A)
print('Salvage value: ', s)
print('Additional producing cost: ', l)
print('Selling prices: ', q)
print('Cost coefficient:', c)

Preorder cost:  [9 9 9 9 9]
Material usage:  [[6 1 2 1 3]
 [3 6 5 2 2]
 [1 8 8 6 1]
 [6 3 7 1 6]
 [2 7 2 8 3]
 [2 4 7 9 2]
 [9 7 3 3 7]
 [2 4 8 4 7]]
Salvage value:  [8 1 4 6 6]
Additional producing cost:  [103 128 112 135 145 103 185 178]
Selling prices:  [1392 1205 1199 1116 1202 1274 1243 1135]
Cost coefficient: [-1289 -1077 -1087  -981 -1057 -1171 -1058  -957]


In [ ]:
m = Container()